# **INTRODUCTION**
In this notebook, we shall be using the BERT transformer to create model to perform  sentiment analysis on our data.First we shall install the dependancies needed in the notebook.

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip install -q transformers

In [ ]:
!pip install tensorflow

In [ ]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
!pip install datasets

We shall then make the relevant imports.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import string
import sklearn
import  torch
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
import tensorflow as tf
#metrics
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
import torch
torch.cuda.is_available()
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
# Hugging Face Dataset
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

from transformers import DataCollatorWithPadding


Lets read our dataset which contains various sentiments from Reddit app.


In [ ]:
df=pd.read_csv('/content/cleaned_data.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,original_text,text,listed_emotions,emotion_count,encoded_labels
0,0,0,That game hurt.,g a m e h u r t,sadness,1,0
1,1,1,"You do right, if you don't care then fuck 'em!",r i g h t c a r e f u c k,neutral,1,2
2,2,2,Man I love reddit.,m a n l o v e r e d d i t,love,1,1
3,3,3,"[NAME] was nowhere near them, he was by the Fa...",n a m e n o w h e r e n e a r f a l c o n,neutral,1,2
4,4,4,Right? Considering it’s such an important docu...,r i g h t c o n s i d e r i n g i m p o r ...,gratitude,1,1


In [ ]:
# Drop rows with missing values in the "text" column
df.dropna(subset=["text"], inplace=True)

From the above dataset, we can see , the original text, which is the text before cleaning, the text that is cleaned (text column).The listed emotions their count and their encoded labels. </br>
We have four types of emotions and their encoded lables, these are as shown below : </br>

1. Negative emotions : label 0.
2. Positive  emotions :label 1.
3. Neutral emotions : label 2.


In [ ]:
df=df.rename(columns={"encoded_labels": "labels", "text":"cleaned_text"})
df.head()

,Unnamed: 0.1,Unnamed: 0,original_text,cleaned_text,listed_emotions,emotion_count,labels
0,0,0,That game hurt.,g a m e h u r t,sadness,1,0
1,1,1,"You do right, if you don't care then fuck 'em!",r i g h t c a r e f u c k,neutral,1,2
2,2,2,Man I love reddit.,m a n l o v e r e d d i t,love,1,1
3,3,3,"[NAME] was nowhere near them, he was by the Fa...",n a m e n o w h e r e n e a r f a l c o n,neutral,1,2
4,4,4,Right? Considering it’s such an important docu...,r i g h t c o n s i d e r i n g i m p o r ...,gratitude,1,1


We shall only collect the columns that we need as we go on, the text and the labels.I know that using the original text raises the question why are we not using the cleaned text to perform the analysis yet we know that we need to preprocess the data before  we feed it to our model?</br>
<font color ='green'>
 **Well the answer to this question is that , the bert model is a deep learning model, so it has been trained to deal with complex data.With bert, every "dirt" if I could call it so is important, every punctuation, everything enables is to be able to give it meaning.I had initially tried modelling this with the cleaned data and I was getting a very low accuracy of about 0.38 and even after I hyperparameter tune I was getting an even lower score and so only when I changed this was I able to get a better score.**</font>

In [ ]:
##Getting the columns we need from the data.
df=df[['original_text','labels']]
df.head()

,original_text,labels
0,That game hurt.,0
1,"You do right, if you don't care then fuck 'em!",2
2,Man I love reddit.,1
3,"[NAME] was nowhere near them, he was by the Fa...",2
4,Right? Considering it’s such an important docu...,1


In [ ]:
df=df.rename(columns={"original_text":"text"})
df.head()

,text,labels
0,That game hurt.,0
1,"You do right, if you don't care then fuck 'em!",2
2,Man I love reddit.,1
3,"[NAME] was nowhere near them, he was by the Fa...",2
4,Right? Considering it’s such an important docu...,1


In [ ]:
df.labels.value_counts()

1    86778
2    55049
0    54834
Name: labels, dtype: int64

From the above , we can see that there is class imbalance between the labels

## 2. CREATING A SIMPLE PIPELINE MODEL.
Below we shall create a simple model that performs sentiment analysis using pipelines that are already pretrained

In [ ]:
#start by getting the specific column that we want to analyze.
data=df.text[0]

In [ ]:
data

'That game hurt.'

In [ ]:
# creating a simple sentiment analysis with pipelines.rs
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
## lets see the sentiments for row 1

sentiment_pipeline(data)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'NEGATIVE', 'score': 0.9996750354766846}]

We can see that our model performs well, hence we should seek to explore other models for this tasks.

In [ ]:
# Step 2: Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
print("Legnth of our training data set",len(train_data))
print("Legnth of our testing data set",len(test_data))

Legnth of our training data set 157328
Legnth of our testing data set 39333


# 3. Tokenization.
If we want to use a bert model, we sahll use a bert tokenizer to tokenize the text column so that we can feed it into our model.

In [ ]:
# instantiate the pretrained tokenizer
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Take a look at the tokenizer
tokenizer


BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

We then convert our dataframe to a huggingframe dataset so that we can be able to easily work with the data

In [ ]:
# Convert pyhton dataframe to Hugging Face arrow dataset
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)


In [ ]:
hg_train_data = hg_train_data.shuffle(seed=42).select([i for i in list(range(10000))])
hg_test_data = hg_test_data.shuffle(seed=42).select([i for i in list(range(2500))])


In [ ]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["text"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")



In [ ]:
# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
# Take a look at the data
print(dataset_train)
print(dataset_test)


Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2500
})


In [ ]:
# Check the first record
dataset_train[0]

{'text': 'In America [...] the poor see themselves not as an exploited proletariat, but as temporarily embarrassed millionaires. - [NAME]',
 'labels': 0,
 '__index_level_0__': 195314,
 'input_ids': [101,
  1130,
  1738,
  164,
  119,
  119,
  119,
  166,
  1103,
  2869,
  1267,
  2310,
  1136,
  1112,
  1126,
  20903,
  5250,
  5765,
  11315,
  1204,
  117,
  1133,
  1112,
  7818,
  10529,
  1550,
  9674,
  1116,
  119,
  118,
  164,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

# 4) MODELLING
### 4.1) MODELLING WITH DATA THAT HAS IMBALANCED CLASS IN THE LABELS COLUMNS

First, we shall craete a model using the data as it is , even though there is imbalanced class then see how the model performs.We are using the bert model, hence we shall instantiate a bert model.

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Since our model is pretrained, we shall create our training arguments and pass them into the trainer then train our model

In [ ]:
!pip install accelerate -U
! pip install -U transformers


In [ ]:
from torch.optim import AdamW
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:

from transformers import EarlyStoppingCallback, IntervalStrategy

Create the training arguments.

In [ ]:
# Set up training arguments

training_args = TrainingArguments(
    output_dir="Chepchumba/autotrain-data-sentiment_analysis",
    logging_dir='Chepchumba/autotrain-data-sentiment_analysis/logs',
    logging_strategy='epoch',
    logging_steps=2000,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    weight_decay=0.01,
    seed=42,
    save_strategy='epoch',
    save_steps=1000,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,

)




In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 619.5 kB/s eta 0:00:00


We shall use  a data collator that helps us to orgarnize and structure our data in a format that is suitable for training the machine model.

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Model performance evaluation
import evaluate

We shall also create a function called compute_metric that will enable us to evaluate the model by checking its accuracy.

In [ ]:
import numpy as np
from datasets import load_metric

import numpy as np
from datasets import load_metric

# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy","f1")
    logits, labels = eval_pred
    probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)





Now that we have the model, lets go ahead and use it to train on our data.

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.939100,0.843202,0.626800


Epoch,Training Loss,Validation Loss,Accuracy
1,0.939100,0.843202,0.626800
2,0.785600,0.846391,0.626400


TrainOutput(global_step=1250, training_loss=0.862377880859375, metrics={'train_runtime': 5237.6736, 'train_samples_per_second': 3.818, 'train_steps_per_second': 0.239, 'total_flos': 328891772160000.0, 'train_loss': 0.862377880859375, 'epoch': 2.0})

In [ ]:
trainer.evaluate(dataset_test)

{'eval_loss': 0.8432015776634216,
 'eval_accuracy': 0.6268,
 'eval_runtime': 190.7431,
 'eval_samples_per_second': 13.107,
 'eval_steps_per_second': 0.823,
 'epoch': 2.0}

Our model has an accuracy of around 0.62, lets go ahead and see how our model performs when we use data with a balanced class of labels.

# MODELLING WITH DATA THAT HAS A BALANCED CLASS OF LABELS.

Our dataset has some class imbalance in the labels column,this could negatively affect our model, hence we should remove the class imbalance.We shall use the RandomUnderSampler as a method to handle class imbalance in this notebook.

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE


In [ ]:
smote = SMOTE(sampling_strategy='auto', random_state=42)


In [ ]:
# We shall use the randomuderSampler because it shall resample our data and do away with class imbalance
## The sampler shall also help us reduce the number of rows we have which is good because it
## saves us time when running our model

# Import the necessary libraries
from imblearn.under_sampling import RandomUnderSampler

# Create the RandomUnderSampler object
under_sampler = RandomUnderSampler(random_state=42)

# Perform random undersampling on the DataFrame
data_resampled, _ = under_sampler.fit_resample(df, df['labels'])

# Now data_resampled will contain the randomly undersampled data


In [ ]:
data_resampled

,text,labels
0,That game hurt.,0
1,"He isn't as big, but he's still quite popular....",0
2,"I wanted to downvote this, but it's not your f...",0
3,That is odd.,0
4,For extra measure tape it right by your crotch...,0
...,...,...
164497,Anyone want adopt a dog while we are at this d...,2
164498,Weebles wobble but they don't fall down.,2
164499,"Unreal Engine is terrible for MMOs, yet devs k...",2
164500,"I was reading up on that, sounds like the long...",2


In [ ]:
data_resampled.labels.value_counts()

0    54834
1    54834
2    54834
Name: labels, dtype: int64

From the above, we can see that our number of rows have reduced and that we have a balance in the labels columns.Know lets go ahead and create the datastes then tokenize the data.We will just follow the steps like we did earlier.

In [ ]:
# Step 2: Split the data into training and testing sets
train_data, test_data = train_test_split(data_resampled, test_size=0.2, random_state=42)

In [ ]:
print("Legnth of our training data set",len(train_data))
print("Legnth of our testing data set",len(test_data))

Legnth of our training data set 131601
Legnth of our testing data set 32901


In [ ]:
# Convert pyhton dataframe to Hugging Face arrow dataset
hg_train_data_2 = Dataset.from_pandas(train_data)
hg_test_data_2 = Dataset.from_pandas(test_data)

In [ ]:
hg_train_data_2 = hg_train_data_2.shuffle(seed=42).select([i for i in list(range(10000))])
hg_test_data_2 = hg_test_data_2.shuffle(seed=42).select([i for i in list(range(2500))])

In [ ]:
# Tokenize the dataset
dataset_train_2 = hg_train_data_2.map(tokenize_dataset)
dataset_test_2 = hg_test_data_2.map(tokenize_dataset)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
# Take a look at the data
print(dataset_train_2)
print(dataset_test_2)

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2500
})


In [ ]:
# Check the first record
dataset_train_2[0]

{'text': "Not a [NAME] fan myself, but there's no reason to comment on his weight or appearance. It only makes your side look bad.",
 'labels': 2,
 '__index_level_0__': 126260,
 'input_ids': [101,
  1753,
  170,
  164,
  151,
  10964,
  2036,
  166,
  5442,
  1991,
  117,
  1133,
  1175,
  112,
  188,
  1185,
  2255,
  1106,
  7368,
  1113,
  1117,
  2841,
  1137,
  2468,
  119,
  1135,
  1178,
  2228,
  1240,
  1334,
  1440,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Now let us train the model with our new datasets and see how it performs.

In [ ]:
# Train the model
trainer_2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_2,
    eval_dataset=dataset_test_2,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
trainer_2.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.951700,0.880115,0.594000
2,0.811300,0.878787,0.597600


TrainOutput(global_step=1250, training_loss=0.881482470703125, metrics={'train_runtime': 8058.8804, 'train_samples_per_second': 2.482, 'train_steps_per_second': 0.155, 'total_flos': 328891772160000.0, 'train_loss': 0.881482470703125, 'epoch': 2.0})

# CONCLUSION.
 We can see that our  second model has a lower accuracy compared to the first model.While there is no reasonable explanation for this, I would say that the best performing model so far is the first model using the data as it is.